# INM702 Neural Network for MNIST

## Mathematical Functions and Derivatives

In [ ]:
#Activation Functions
import numpy as np
#Input and Hidden Layers
@np.vectorize
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

@np.vectorize
def d_sigmoid(x):
    pass 

@np.vectorize
def relu(x):
    pass

@np.vectorize
def d_relu(x):
    pass

#Output Layer
@np.vectorize
def softmax(x):
    pass
    
#Loss Functions
@np.vectorize
def cross_entropy_loss(x):
    pass

@np.vectorize
def d_loss(x):
    pass

#Truncated Normal Distribution for Initialising weights
from scipy.stats import truncnorm
def trunc_norm(mean = 0, sd = 1, lower = -1, upper = 1):
    return truncnorm((lower - mean) / sd, (upper - mean) / sd, loc=mean, scale=sd)
    

## Neural Network Class

In [ ]:
class NeuralNetwork:
    def __init__(self, hidden_layer_sizes = [128,128], learning_rate = 0.001 , activation_fn = 'r', dropout_rate = 0):
        self.layer_sizes = [28*28] + hidden_layer_sizes + [10]
        self.lr = learning_rate
        self.dr = dropout_rate
        
        #activation_fn parameter takes 'r'(default) or 's' to choose ReLu or Sigmoid for the activation function
        if activation_fn == 's':
            self.activation = sigmoid
            self.d_activation = d_sigmoid
        else:
            self.activation = relu
            self.d_activation = d_relu
            
        self.initialise_weights()
    
    def initialise_weights(self):
        pass
    
    def forward_pass(self, x):
        pass
    
    def back_prop(self, y):
        pass
    
    def train(self, x_train, y_train, epochs = 5):
        pass
    
    def test(self, x_test):
        pass
            
        